In [7]:
import os
from fragment_MOFs_for_pormake import make_MOF_fragments
import pandas as pd
import warnings
from ase.io import read, write
import shutil
import numpy as np
import glob
from pymatgen.core.structure import Structure
from pymatgen.analysis import structure_matcher
import pymatgen.core as mg
from tqdm import tqdm

In [2]:
qmofid = pd.read_excel("../qmof/qmofid.xlsx")
qmofid

,qmof_id,name,info.formula,info.formula_reduced,info.mofid.mofkey,info.mofid.smiles_nodes,info.mofid.smiles_linkers,info.mofid.topology,info.natoms,info.pld,info.lcd,info.density,info.volume,info.symmetry.spacegroup,info.symmetry.spacegroup_number,info.symmetry.spacegroup_crystal,info.symmetry.pointgroup,info.source,outputs.pbe.energy_total,outputs.pbe.bandgap
0,qmof-830ed1c,ABAVIJ_FSR,Co4C48H32N8O16,CoC12H8N2O4,Co.TWBYWOBDOCUKOW.MOFkey-v1.rtl,['[Co]'],['[O-]C(=O)c1ccncc1'],rtl,108,2.36128,4.21176,1.557644,1292.643180,Cc,9,monoclinic,m,CSD,-759.996078,0.345448
1,qmof-5bd4a24,ABAVOP_FSR,Co4C48H32N8O16,CoC12H8N2O4,Co.TWBYWOBDOCUKOW.MOFkey-v1.rtl,['[Co]'],['[O-]C(=O)c1ccncc1'],rtl,108,2.14542,3.27957,1.616139,1245.856916,P2_1/c,14,monoclinic,2/m,CSD,-760.103362,0.342645
2,qmof-d2e3fe6,ABAZAF_FSR,Zn4C104H76N12O20,ZnC26H19N3O5,Zn.MGFJDEHFNMWYBD.QNVNLUSHGRBCLO.MOFkey-v1.dia,['[Zn]'],"['Oc1cc(cc(c1)C(=O)[O-])C(=O)[O-]', 'n1ccc(cc1...",dia,216,1.14158,2.11158,1.526090,2258.265282,P2_1/c,14,monoclinic,2/m,CSD,-1476.754182,1.810322
3,qmof-f364177,ABAZAF01_FSR,Zn4C104H76N12O20,ZnC26H19N3O5,Zn.MGFJDEHFNMWYBD.QNVNLUSHGRBCLO.MOFkey-v1.dia,['[Zn]'],"['Oc1cc(cc(c1)C(=O)[O-])C(=O)[O-]', 'n1ccc(cc1...",dia,216,1.13150,2.09266,1.524741,2260.263800,P2_1/c,14,monoclinic,2/m,CSD,-1476.957225,1.500148
4,qmof-de2f502,ABEREH01_FSR,Cu4C8H16O12,CuC2H4O3,Cu.BDAGIHXWWSANSR.GRVDJDISBSALJP.MOFkey-v1.hcb,['[Cu]'],"['C[O]', '[O-]C=O']",hcb,40,0.69357,1.63699,2.466120,375.986316,P-1,2,triclinic,-1,CSD,-227.092030,0.360798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,qmof-d0d26c4,ZUJGET_FSR,Ni4C72H56N8O22,Ni2C36H28N4O11,Ni.RXOHFPCZGPKIRD.XLLIUDMGQNPRJS.MOFkey-v1.acs,['[OH2][Ni][OH2][Ni][OH2]'],"['[N]([CH]c1ccncc1)[N][CH]c1ccncc1', '[O-]C(=O...",acs,162,2.03849,5.52104,1.231932,2183.660172,C2/c,15,monoclinic,2/m,CSD,-1104.096323,0.854835
4319,qmof-6fc44b7,ZUJGIX_FSR,Co4C100H72N8O22,Co2C50H36N4O11,Co.PIDBTEHSRNNXGY.MOFkey-v1.pts,['[Co][OH2][Co]'],['[O-]C(=O)c1cc(OCc2ccc(cc2)c2ccc(cc2)COc2cc(c...,pts,206,1.23849,4.49148,1.439374,2276.637007,C2/c,15,monoclinic,2/m,CSD,-1424.568324,0.419735
4320,qmof-f527291,ZUQPIK_FSR,Cu4C64H64I4S8,CuC16H16IS2,Cu.CUWSXVJIPZWUGC.MOFkey-v1.sql,['I[Cu][Cu]I'],['S1Cc2ccc(cc2)CSCc2ccc(C1)cc2'],sql,144,3.51453,5.23076,1.713334,1794.462462,Cmc2_1,36,orthorhombic,mm2,CSD,-879.427700,2.163370
4321,qmof-5206c61,ZURQOS_freeONLY,CdNiC12H10N6,CdNiC12H10N6,NiCd.FKZGFAZIEVONAY.JEVCWSUVFOYBFI.MOFkey-v1.fsc,"['[Cd]', '[Ni]']","['[C]#N', '[NH]Cc1ccc(cc1)C[NH]']",fsc,30,3.60748,4.57674,1.315703,516.640739,P-1,2,triclinic,-1,CoRE,-197.744737,1.130483


In [3]:
def ase_format(root_dir):
    cifs = os.listdir(root_dir)
    cifs.sort()
    for cif in cifs:
        try:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                mof = read(os.path.join(root_dir, cif))
                write(os.path.join(root_dir, cif), mof)
                print('ASE reading: ' + cif)
        except:
            print('ASE reading fail: ' + cif)

In [6]:
def conv2qmof(root_dir):
    cifs = os.listdir(root_dir)
    cifs.sort()
    mofs = []
    for cif in cifs:
        # try:
            mof_temp = Structure.from_file(os.path.join(root_dir, cif),primitive=True)
            mofs.append(mof_temp)
            print('trans to primituve cell: ' + cif)
    sm = structure_matcher.StructureMatcher(primitive_cell=True)
    groups = sm.group_structures(mofs)
    for group in groups:
        mof_temp = group[0]
        mof_temp.to(filename=os.path.join(root_dir,cif))
        # except:
             # print('pymatgen reading fail: ' + cif)

In [25]:
for name in qmofid["name"]:
    source_file = '../qmof/unrelaxed/' + name + ".cif"
    destination_file = './qmofid/cif/' + name + ".cif"
    try:
        shutil.copyfile(source_file, destination_file)
    except:
        print(name)

BOQTAD_FSR
FISQEG_FSR
QEPZOB_FSR
KUTNEU_FSR
DANTUJ_FSR
POJTOY01_FSR
TOWLOI_FSR
NAZBAT_FSR
VACFOV01_FSR
EMAXOH_FSR
DAFZAO_FSR
BONMAT_FSR
BONMEX_FSR
LUPVEZ_FSR
EBAMOL_FSR
KEVBUK01_FSR
WAWGOQ_FSR
ELESEU_FSR
HIDMEO_FSR
LOSGIL_FSR


In [4]:
fragmentation_directory = './coremof/FSR_clean/'

In [8]:
ase_format(fragmentation_directory+'cif/')
conv2qmof(fragmentation_directory+'cif/')

trans to primituve cell: ABAVIJ_FSR.cif
trans to primituve cell: ABAVOP_FSR.cif
trans to primituve cell: ABAZAF01_FSR.cif
trans to primituve cell: ABAZAF_FSR.cif
trans to primituve cell: ABEREH01_FSR.cif
trans to primituve cell: ABIWAL_FSR.cif
trans to primituve cell: ABIWEP01_FSR.cif
trans to primituve cell: ACAJIZ_freeONLY.cif
trans to primituve cell: ACAJOF_FSR.cif
trans to primituve cell: ACAKEX_FSR.cif
trans to primituve cell: ACATAB_FSR.cif
trans to primituve cell: ACAZAG_FSR.cif
trans to primituve cell: ACEKOK_FSR.cif
trans to primituve cell: ACIGEB_FSR.cif
trans to primituve cell: ACIRIP_FSR.cif
trans to primituve cell: ACODAA_FSR.cif
trans to primituve cell: ACOGAB_freeONLY.cif
trans to primituve cell: ACOGEF_freeONLY.cif
trans to primituve cell: ACOLIP_FSR.cif
trans to primituve cell: ACOLUB_FSR.cif
trans to primituve cell: ACUBOP01_FSR.cif
trans to primituve cell: ACUBOP_FSR.cif
trans to primituve cell: ACUFUB_FSR.cif
trans to primituve cell: ACUGAH_FSR.cif
trans to primituv

/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 1 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


trans to primituve cell: AFITIT_FSR.cif
trans to primituve cell: AFITUF_freeONLY.cif
trans to primituve cell: AFIXAO_freeONLY.cif
trans to primituve cell: AFOROD_FSR.cif
trans to primituve cell: AFOSIY_FSR.cif
trans to primituve cell: AFOTUL_FSR.cif
trans to primituve cell: AFOVAT_FSR.cif
trans to primituve cell: AFOVEX_FSR.cif
trans to primituve cell: AFOVOH_FSR.cif
trans to primituve cell: AFUKAP_FSR.cif
trans to primituve cell: AGAWOU_FSR.cif
trans to primituve cell: AGEDIC_FSR.cif
trans to primituve cell: AGOSOE_FSR.cif
trans to primituve cell: AGOZED01_FSR.cif
trans to primituve cell: AGOZED_FSR.cif
trans to primituve cell: AGULIY_FSR.cif
trans to primituve cell: AGUTEC_FSR.cif
trans to primituve cell: AGUWEG01_FSR.cif
trans to primituve cell: AHAHIB_FSR.cif
trans to primituve cell: AHEMOQ_FSR.cif
trans to primituve cell: AHINIP_FSR.cif
trans to primituve cell: AHOMOB01_FSR.cif
trans to primituve cell: AHXCUD10_FSR.cif
trans to primituve cell: AJABIX01_FSR.cif


/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 9 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


trans to primituve cell: AJABOD_FSR.cif
trans to primituve cell: AJAQUY_FSR.cif
trans to primituve cell: AJAXOY_FSR.cif
trans to primituve cell: AJEKIL_FSR.cif
trans to primituve cell: AJEWUH_FSR.cif
trans to primituve cell: AJIFEF_FSR.cif
trans to primituve cell: AJIJEI01_FSR.cif
trans to primituve cell: AJIJEI_FSR.cif
trans to primituve cell: AJIJIN_FSR.cif
trans to primituve cell: AJOFAI_FSR.cif
trans to primituve cell: AJUNOK_FSR.cif
trans to primituve cell: AKAGIE_FSR.cif
trans to primituve cell: AKAROT01_FSR.cif
trans to primituve cell: AKAROT_FSR.cif
trans to primituve cell: AKAXOA_FSR.cif
trans to primituve cell: AKAYIV_FSR.cif
trans to primituve cell: AKAZOD_FSR.cif
trans to primituve cell: AKEDIF_freeONLY.cif
trans to primituve cell: AKEGON_FSR.cif
trans to primituve cell: AKEGUT_FSR.cif
trans to primituve cell: AKESIS_FSR.cif
trans to primituve cell: AKIZAX_FSR.cif
trans to primituve cell: AKIZEB_FSR.cif
trans to primituve cell: ALEQIS_FSR.cif
trans to primituve cell: ALIDUU

...

trans to primituve cell: RUYTOW_FSR.cif
trans to primituve cell: RUZBUL_FSR.cif
trans to primituve cell: RUZCAS_FSR.cif
trans to primituve cell: RUZCEW_FSR.cif
trans to primituve cell: RUZCOG_FSR.cif
trans to primituve cell: RUZLAA_FSR.cif
trans to primituve cell: SABVOH01_freeONLY.cif
trans to primituve cell: SABWAU01_FSR.cif
trans to primituve cell: SACCAD_FSR.cif
trans to primituve cell: SACNUI_FSR.cif
trans to primituve cell: SADLOA_FSR.cif
trans to primituve cell: SADMIV_freeONLY.cif
trans to primituve cell: SADMUH_SL.cif
trans to primituve cell: SAFZIL_FSR.cif
trans to primituve cell: SAHWEG_FSR.cif
trans to primituve cell: SAHYOS_FSR.cif
trans to primituve cell: SAJSAY_FSR.cif
trans to primituve cell: SAKDEP_freeONLY.cif
trans to primituve cell: SALLAT_FSR.cif
trans to primituve cell: SALRON_FSR.cif
trans to primituve cell: SAMTOS_FSR.cif
trans to primituve cell: SAMTUY_FSR.cif
trans to primituve cell: SAMZIS_FSR.cif
trans to primituve cell: SAQGAU_FSR.cif
trans to primituve cel

/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 7 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


trans to primituve cell: SERKAC_FSR.cif
trans to primituve cell: SESHEG_FSR.cif
trans to primituve cell: SETCUQ_FSR.cif
trans to primituve cell: SETPEO_freeONLY.cif
trans to primituve cell: SETQUF_FSR.cif
trans to primituve cell: SETSOB_freeONLY.cif
trans to primituve cell: SEVROC_FSR.cif
trans to primituve cell: SEWWUN_FSR.cif
trans to primituve cell: SEYVEY01_FSR.cif
trans to primituve cell: SEYVIC01_FSR.cif
trans to primituve cell: SIBDOY_FSR.cif
trans to primituve cell: SICGIW_FSR.cif
trans to primituve cell: SIFLEY_FSR.cif
trans to primituve cell: SIGMED_FSR.cif
trans to primituve cell: SIGXUE_FSR.cif
trans to primituve cell: SIMXOE_FSR.cif
trans to primituve cell: SIMZOD02_FSR.cif
trans to primituve cell: SINKIL_FSR.cif
trans to primituve cell: SINSUG_FSR.cif
trans to primituve cell: SINZIA_FSR.cif
trans to primituve cell: SIPHUV_FSR.cif
trans to primituve cell: SIRVEX_FSR.cif
trans to primituve cell: SISGIM_FSR.cif
trans to primituve cell: SISGOS_FSR.cif
trans to primituve cell:

/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 22 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


trans to primituve cell: VONBUX02_FSR.cif
trans to primituve cell: VONCUY_FSR.cif
trans to primituve cell: VOPHUF_FSR.cif
trans to primituve cell: VOQBUZ_FSR.cif
trans to primituve cell: VOXQIK_FSR.cif
trans to primituve cell: VOYBOC_FSR.cif
trans to primituve cell: VOYLOM_FSR.cif
trans to primituve cell: VUJSID_FSR.cif
trans to primituve cell: VUKHEP01_FSR.cif
trans to primituve cell: VUKHEP_FSR.cif
trans to primituve cell: VUKHOZ_freeONLY.cif
trans to primituve cell: VUKMUJ05_FSR.cif
trans to primituve cell: VUKRUQ_FSR.cif
trans to primituve cell: VUKXUW_FSR.cif
trans to primituve cell: VUMQEA_FSR.cif
trans to primituve cell: VUMYEJ_FSR.cif
trans to primituve cell: VUNDOZ_FSR.cif
trans to primituve cell: VUNJEU_FSR.cif
trans to primituve cell: VUNPOL_FSR.cif
trans to primituve cell: VUPQON_FSR.cif
trans to primituve cell: VURMOL_FSR.cif
trans to primituve cell: VUSNUT_FSR.cif
trans to primituve cell: VUTXEP01_FSR.cif
trans to primituve cell: VUWTAL_FSR.cif
trans to primituve cell: VU

In [8]:
log_list = []
for cif_file in tqdm(os.listdir(fragmentation_directory)): # assumes that all of the cifs are in a directory called cif inside of the parent directory.
    # print('now on', cif_file)  
    try:
        return_code = make_MOF_fragments(fragmentation_directory+cif_file,
                                         path=fragmentation_directory+'/',
                                         xyzpath=fragmentation_directory+'/xyz/'+cif_file.replace('cif','xyz'))
        if return_code == None:
            return_code = 'good'
        log_list.append({'cif':cif_file,'return_code':return_code})
    except:
        print(cif_file,"fail")

df = pd.DataFrame(log_list)
df.to_csv(fragmentation_directory+'/fragment_status.csv')

  0%|                                                                                             | 0/1962 [00:00<?, ?it/s]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is image0)



('cell vectors: ', 'alpha, beta, gamma = 82.102, 90.0, 90.0')


  0%|                                                                                   | 1/1962 [00:07<3:59:06,  7.32s/it]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is image0)



('cell vectors: ', 'alpha, beta, gamma = 86.6591, 86.6591, 45.7913')


  0%|                                                                                   | 2/1962 [00:07<1:47:22,  3.29s/it]

...

('cell vectors: ', 'alpha, beta, gamma = 90.0, 90.0, 90.0')
structure is interpenetrated


100%|███████████████████████████████████████████████████████████████████████████████▉| 1960/1962 [3:59:31<00:41, 20.95s/it]

('cell vectors: ', 'alpha, beta, gamma = 90.0, 90.0, 90.0')
structure is interpenetrated


100%|███████████████████████████████████████████████████████████████████████████████▉| 1961/1962 [4:00:49<00:34, 34.56s/it]==============================
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is image0)

100%|████████████████████████████████████████████████████████████████████████████████| 1962/1962 [4:00:49<00:00,  7.36s/it]

('cell vectors: ', 'alpha, beta, gamma = 73.674, 90.0, 90.0')
structure is interpenetrated


In [19]:
def should_delete_line(previous_line, current_line, line_index):
    if line_index <= 2:
        return False
    if len(previous_line.split()) == 4 and len(current_line.split()) <= 3:
        return True
    return False

In [8]:
periodic_table_symbols = [
    'X','H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg',
    'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr',
    'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
    'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
    'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf',
    'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po',
    'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm',
    'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs',
    'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og'
    ]

In [34]:
names = ["ABIWAL_FSR"]

In [42]:
for name in qmofid["name"]:
# for name in names:
    lines_to_keep = []
    previous_line = None

    try:
        # linker data
        lines_to_keep = []
        line_index = 1
        with open("./qmofid/linkers/" + name + '_linker_0.xyz', 'r') as file:
            for line in file:
                if previous_line and should_delete_line(previous_line, line, line_index):
                    continue 
                else:
                    lines_to_keep.append(line) 
                line_index += 1
                previous_line = line 
        with open("./qmofid/linkers/" + name + '_linker_0_4ase.xyz', 'w') as file:
            file.writelines(lines_to_keep)
        file.close()
        atoms1 = read("./qmofid/linkers/" + name + '_linker_0_4ase.xyz')
        pos1 = atoms1.positions
        atom1 = atoms1.get_chemical_symbols()
        linkerinfo = []
        for (i,ele1) in enumerate(atom1):
            atom_index1 = periodic_table_symbols.index(ele1)
            x1 = pos1[i][0]
            y1 = pos1[i][1]
            z1 = pos1[i][2]
            atom_per1 = [atom_index1,x1,y1,z1]
            linkerinfo.append(atom_per1)
        np.save("./qmofid/input/" + name + "_linker.npy", linkerinfo)
    
        # sbu data
        lines_to_keep = []
        line_index = 1
        with open("./qmofid/sbus/" + name + '_sbu_0.xyz', 'r') as file:
            for line in file:
                if previous_line and should_delete_line(previous_line, line, line_index):
                    continue 
                else:
                    lines_to_keep.append(line) 
                line_index += 1
                previous_line = line 
        with open("./qmofid/sbus/" + name + '_sbu_0_4ase.xyz', 'w') as file:
            file.writelines(lines_to_keep)
        file.close()
        atoms2 = read("./qmofid/sbus/" + name + '_sbu_0_4ase.xyz')
        pos2 = atoms2.positions
        atom2 = atoms2.get_chemical_symbols()
        sbuinfo = []
        for (i,ele2) in enumerate(atom2):
            atom_index2 = periodic_table_symbols.index(ele2)
            x2 = pos2[i][0]
            y2 = pos2[i][1]
            z2 = pos2[i][2]
            atom_per2 = [atom_index2,x2,y2,z2]
            sbuinfo.append(atom_per2)
        np.save("./qmofid/input/" + name + "_sbu.npy", sbuinfo)
    
        #topology data
        topo_name = qmofid[qmofid["name"]==name]["info.mofid.topology"]
        topo_struc = "./qmofid/topo/" + topo_name.values[0] + ".cif"
        atoms3 = read(topo_struc)
        pos3 = atoms3.positions
        atom3 = atoms3.get_chemical_symbols()
        topoinfo = []
        for (i,ele3) in enumerate(atom3):
            atom_index3 = periodic_table_symbols.index(ele3)
            x3 = pos3[i][0]
            y3 = pos3[i][1]
            z3 = pos3[i][2]
            atom_per3 = [atom_index3,x3,y3,z3]
            topoinfo.append(atom_per3)
        np.save("./qmofid/input/" + name + "_topo.npy", topoinfo)
    
        # atom data
        atoms4 = read("./qmofid/cif/" + name +".cif")
        pos4 = atoms4.positions
        atom4 = atoms4.get_chemical_symbols()
        atominfo = []
        for (i,ele4) in enumerate(atom4):
            atom_index4 = periodic_table_symbols.index(ele4)
            x4 = pos4[i][0]
            y4 = pos4[i][1]
            z4 = pos4[i][2]
            atom_per4 = [atom_index4,x4,y4,z4]
            atominfo.append(atom_per4)
        np.save("./qmofid/input/" + name + "_atom.npy", atominfo)
    
        # tar: pos & cell
        atoms5 = read("../qmof/relaxed/" + name +".cif")
        pos5 = atoms5.positions
        atom5 = atoms5.get_chemical_symbols()
        postarinfo = []
        for (i,ele5) in enumerate(atom5):
            x5 = pos5[i][0]
            y5 = pos5[i][1]
            z5 = pos5[i][2]
            atom_per5 = [x5,y5,z5]
            postarinfo.append(atom_per5)
        np.save("./qmofid/output/" + name + "_pos.npy", postarinfo)
        
        structure = mg.Structure.from_file("../qmof/relaxed/" + name +".cif")
        lattice = structure.lattice.matrix
        np.save("./qmofid/output/" + name + "_cell.npy", lattice)
    except:
        print(name)

ABAVIJ_FSR
ABAVOP_FSR
ABAZAF_FSR
ABAZAF01_FSR
ABEREH01_FSR
ABIWEP01_FSR
ACAJIZ_freeONLY
ACAKEX_FSR
ACATAB_FSR
ACAZAG_FSR
ACEKOK_FSR
ACIRIP_FSR
ACODAA_FSR
ACUBOP_FSR
ACUBOP01_FSR
ACUFUB_FSR
ACUGAH_FSR
ACUGEL_FSR


/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


ADAVEJ01_FSR
ADEGIA01_FSR
ADINOT_FSR
ADIQEL_FSR
ADIZUJ_FSR
ADOXUP_FSR
AFECIY01_FSR
AFEYUI_FSR
AFITIT_FSR
AFITUF_freeONLY
AFIXAO_freeONLY
AFOTUL_FSR
AFOVAT_FSR
AFOVEX_FSR
AFOVOH_FSR
AGEDIC_FSR
AGOSOE_FSR
AGOZED_FSR
AGOZED01_FSR
AGULIY_FSR
AGUTEC_FSR
AGUWEG01_FSR
AHEMOQ_FSR
AHOMOB01_FSR
AJABIX01_FSR
AJABOD_FSR
AJAQUY_FSR
AJAXOY_FSR


/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 1 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


AJEWUH_FSR
AJIFEF_FSR
AJIJEI_FSR
AJIJEI01_FSR
AJIJIN_FSR
AKAGIE_FSR
AKAROT_FSR
AKAROT01_FSR
AKEGON_FSR
AKEGUT_FSR
AKESIS_FSR
ALEQIS_FSR
ALIDUU_FSR
ALIMIT_FSR
ALONOE_FSR
ALUJUN_FSR
ALUKAU_FSR
ALUKOI_FSR
ALUKUO_FSR
ALULAV_FSR
AMAFIE_FSR
AMAFOK_freeONLY
AMAGAX_freeONLY
AMBZAG10_FSR
AMIHOV01_FSR
AMIJAJ_FSR
AMIMUE_FSR
AMINEP_FSR
AMODUC_freeONLY
AMORUQ_FSR
AMUCAN_FSR
ANAWOD_FSR
ANAYUL_FSR
ANEHOS_FSR
ANEQUH_FSR
ANODEN_FSR
APARAM_freeONLY
APASAN_FSR
APAXOF_FSR
AQIROJ_FSR
AQODOA_freeONLY
AQOWIN_FSR
ARAHIM01_freeONLY
ARAJEK_FSR
ARAJIO_FSR
ARAJOU_FSR
ARAJUA01_FSR
ARAKAH_FSR
ARATIY_FSR
AREQUJ_FSR
ARIBOS_FSR
ARODUH_FSR
AROTOQ_FSR
ARUTAK_FSR
ARUTEO_FSR
ASAXID_FSR
ASAYOJ_FSR
ASOWOU_FSR
ASUCAS_FSR
ASUQIO_FSR
ASUQIO03_FSR
ATEMAP_FSR
ATEMET_FSR
ATEYAZ_FSR
ATIBOU_FSR
ATIBUA_FSR
ATIXAE_freeONLY
ATOGIZ_FSR
ATOKIE_FSR
ATOKOK_FSR
ATOXEN_FSR
ATULIM_freeONLY
AVAQIX01_FSR
AVATEY01_FSR
AVEMIY_FSR
AVUFED_FSR
AVUJOQ_FSR
AWAWEB_FSR
AWAWIF_FSR
AWEBIP_FSR
AWEJAO_FSR
AWIBAL_FSR
AWIBEP_FSR
AWUBOK_FSR
AXAQAT02_FSR
AXILO

...

KAGKUB_FSR
KAKSAT01_FSR
KAKTIC01_FSR
KAPPID_FSR
KAPYEH01_FSR
KASLUO_FSR
KASPOL_freeONLY
KAXKAY_FSR
KAXXIS_FSR
KBZILT01_FSR
KEDNUE_FSR
KEHWAY01_FSR
KEKLEU_FSR
KEKLIY_FSR
KEKRIE_FSR
KEKWIH_FSR
KELXAB_FSR
KENJUK_FSR
KEQHOG_FSR
KERGIY_FSR
KERXAJ_FSR
KESRUY_FSR
KETNON_FSR
KEVBUK01_FSR
KEVLAA_FSR
KEVLEE_FSR
KEWXIV_FSR
KEXDUN_FSR
KEXFAV_FSR
KEXNEJ_FSR
KEYFAX_FSR
KEYSEO_FSR
KHACET02_FSR
KHMALA03_FSR
KICQUL_FSR
KIFCOR_FSR
KIGKUI_FSR
KIGKUI01_FSR
KIGMIY_FSR
KIGTEA_FSR
KIHDUD_FSR
KIHVOP_FSR
KIJHUI01_FSR
KIKBIR_FSR
KIKDAL_freeONLY
KIKDOZ_freeONLY
KIKDUF_freeONLY
KIMXOV_FSR
KIMYEM_FSR
KIMYIQ_FSR
KINDIW_FSR
KINDOC_FSR
KIPZEQ_FSR
KIQHOH01_FSR
KIRKAZ_freeONLY
KITGUQ_FSR
KITHEB_FSR
KITHIF_FSR
KITHUR_FSR
KIVNIO_FSR
KIVYUK_FSR
KIWSAL_FSR
KIWTAM_FSR
KIXBOL_FSR
KIYFAC_FSR
KOBGOY01_FSR
KOBGOY02_FSR
KOBPAV_FSR
KOFPAY_FSR
KOGNAW_FSR
KOLWEO_freeONLY
KOMMII_FSR
KONYOB_freeONLY
KOQJEH_FSR
KOTCUT_FSR
KOTDEE_FSR
KOTDII_FSR
KOTDOO_FSR
KOXJIT_FSR
KUCFUM_FSR
KUCXUC_FSR
KUFSUB_FSR
KUHDEX_FSR
KULJEI_FSR
KULRIT_FSR
KUMB

/root/anaconda3/envs/qmof/lib/python3.9/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(


POSZED_FSR
POTGUB_FSR
POTHAI_FSR
POVFUE01_FSR
POWVIH_FSR
POXLOF01_FSR
POYPAW_freeONLY
PTRPHT02_FSR
PUFTUH_FSR
PUFXUL_FSR
PUHNAI01_FSR
PUHRUG_FSR
PUHWAS_FSR
PUNZEE_FSR


/root/anaconda3/envs/qmof/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


PUQHAL01_FSR
PUQHEP_freeONLY
PUQHIT_SL
PUQTEB01_FSR
PURJIX_FSR
PURROK_FSR
PURSAX_FSR
QACCOP_FSR
QADDUX_FSR
QAFGAI_FSR
QAFGEM_FSR
QAFNAO01_FSR
QAGWEC_FSR
QAHKUH_FSR
QAKBUZ_FSR
QAKDIP_FSR
QALGUH_SL
QAMSIJ_FSR
QANBAJ_FSR
QANYIQ_FSR
QARCET_FSR
QARVUC_FSR
QARWAJ_FSR
QARWEN_FSR
QARYUE_FSR
QATROU_FSR
QAVDIC01_FSR
QAVFAW_FSR
QAVWAN_freeONLY
QAVWUH_FSR
QAVZEU_FSR
QAVZIY_FSR
QAVZOE_FSR
QAWCEY_FSR
QAWCOI_FSR
QAWJOP01_FSR
QAXBOH_FSR
QAXCAU_FSR
QECDAF_FSR
QECPOG_FSR
QEDMAQ_FSR
QEGNOH_FSR
QEJYIP_freeONLY
QEJZIR01_FSR
QEKYOX_FSR
QEKYUD_FSR
QEKZEO_FSR
QENHIA_FSR
QENHOG_FSR
QEPZOB_FSR
QEWMIP_FSR
QEXHIN_FSR
QEYWUN_FSR
QEZBAZ_FSR
QIDDAI02_FSR
QIFMAT_FSR
QIFMIB_FSR
QIHXIQ_FSR
QIJWOY_FSR
QIKRUZ_FSR
QILRIO_FSR
QIMBIZ_FSR
QIMNEH_FSR
QIPPUD_FSR
QIQREN_FSR
QIRGED_FSR
QISJOR_FSR
QISXEX_FSR
QITLEM_FSR
QIVMOX01_FSR
QIVXIF_FSR
QIVYUR_FSR
QIVZEC_FSR
QIWBAB_FSR
QIYSOH_FSR
QOBHEU_FSR
QOBHEU01_FSR
QOBHIY_FSR
QOBZUD_FSR
QOFLAA_freeONLY
QOGXEQ_FSR
QOHTOX_FSR
QOHTUD_freeONLY
QOKCID_FSR
QOKHIH_FSR
QOKHON_FSR
QOLKUX_FSR
QO

In [36]:
np.load("./qmofid/input/" + name + "_linker.npy")

array([[ 6.      ,  2.578089, 11.573968,  5.64574 ],
       [ 6.      ,  2.924287, 10.651123,  6.605002],
       [ 6.      ,  1.876614, 12.058432,  8.2566  ],
       [ 6.      ,  1.530416, 12.981277,  7.297338],
       [ 1.      ,  2.809702, 11.422721,  4.757381],
       [ 1.      ,  1.645001, 12.209679,  9.144959],
       [ 6.      ,  2.561734, 10.911079,  7.917383],
       [ 6.      ,  1.892969, 12.721321,  5.984957],
       [ 1.      ,  2.785407, 10.299   ,  8.581915],
       [ 1.      ,  1.669295, 13.3334  ,  5.320426],
       [ 0.      ,  3.31966 , 10.036123,  6.437803],
       [ 0.      ,  1.135043, 13.596277,  7.464537]])

In [37]:
np.load("./qmofid/input/" + name + "_sbu.npy")

array([[ 4.8000000e+01, -1.9027990e+00,  5.9081000e+00,  6.9511700e+00],
       [ 8.0000000e+00, -6.5203800e-01,  7.4501140e+00,  8.1606740e+00],
       [ 8.0000000e+00, -4.2044480e+00,  3.4196080e+00,  4.0886780e+00],
       [ 8.0000000e+00,  3.9885100e-01,  8.3965920e+00,  9.8136620e+00],
       [ 6.0000000e+00, -3.8190050e+00,  8.6222810e+00,  7.0651690e+00],
       [ 7.0000000e+00, -3.1823890e+00,  7.6911650e+00,  6.2935890e+00],
       [ 6.0000000e+00, -3.5487960e+00,  7.9499390e+00,  5.0701830e+00],
       [ 7.0000000e+00, -4.3721240e+00,  9.0063080e+00,  5.0090130e+00],
       [ 6.0000000e+00, -4.5634550e+00,  9.4494150e+00,  6.2782970e+00],
       [ 1.0000000e+00, -3.7527840e+00,  8.6837250e+00,  7.9910650e+00],
       [ 6.0000000e+00,  1.3408000e-02,  3.1939190e+00,  6.8371710e+00],
       [ 7.0000000e+00, -6.2320800e-01,  4.1250350e+00,  7.6087510e+00],
       [ 6.0000000e+00, -2.5680100e-01,  3.8662610e+00,  8.8321570e+00],
       [ 7.0000000e+00,  5.6652600e-01,  2.8098920e

In [38]:
np.load("./qmofid/input/" + name + "_topo.npy")

array([[23.,  0.,  0.,  0.],
       [23.,  0., 10.,  0.],
       [23., 10.,  0.,  0.],
       [23., 10., 10.,  0.]])

In [39]:
np.load("./qmofid/input/" + name + "_atom.npy")

array([[ 4.80000000e+01, -1.90279859e+00,  5.90810000e+00,
         6.95117005e+00],
       [ 4.80000000e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 6.00000000e+00,  2.57808874e+00,  1.15739679e+01,
         5.64574032e+00],
       [ 6.00000000e+00,  3.77941266e+00,  5.66586790e+00,
         1.30542974e+00],
       [ 6.00000000e+00,  1.87661407e+00,  2.42232100e-01,
         8.25659979e+00],
       [ 6.00000000e+00,  6.75290147e-01,  6.15033210e+00,
         1.25969104e+01],
       [ 1.00000000e+00,  2.80970198e+00,  1.14227205e+01,
         4.75738078e+00],
       [ 1.00000000e+00,  3.54779942e+00,  5.51462054e+00,
         2.19378927e+00],
       [ 1.00000000e+00,  1.64500083e+00,  3.93479460e-01,
         9.14495932e+00],
       [ 1.00000000e+00,  9.06903387e-01,  6.30157946e+00,
         1.17085508e+01],
       [ 6.00000000e+00,  2.92428665e+00,  1.06511227e+01,
         6.60500179e+00],
       [ 6.00000000e+00,  3.43321476e+00,  4.74302268e+00,
      

In [40]:
np.load("./qmofid/output/" + name + "_cell.npy")

array([[ 8.03105163e+00,  0.00000000e+00, -2.18092200e+00],
       [ 1.90834578e-15,  1.18669169e+01,  7.26639088e-16],
       [ 0.00000000e+00,  0.00000000e+00,  1.44138000e+01]])

In [41]:
np.load("./qmofid/output/" + name + "_pos.npy")

array([[-1.88870143,  5.93345998,  6.95501364],
       [-3.77741963,  0.        , 13.91001239],
       [ 2.91117401, 11.47734996,  4.58868034],
       [ 3.5220295 ,  5.54392001,  2.36631299],
       [ 1.63331053,  0.38955006,  9.32135292],
       [ 1.02248747,  6.32305003, 11.54367715],
       [ 2.85778753, 10.21509995,  8.71124753],
       [-0.20199922,  4.28169005, 12.15377989],
       [ 1.68669697,  1.65179995,  5.19878559],
       [ 4.74649865,  7.58527001,  1.75623653],
       [ 4.64912542,  8.67898998,  8.15399661],
       [-1.99332743,  2.74552004, 12.71103345],
       [-0.10462599,  3.18792997,  5.75601982],
       [ 6.53781551,  9.12137998,  1.19894875],
       [ 5.12185601,  7.43483999,  4.16602098],
       [ 1.31134229,  1.50138998,  2.78899154],
       [-0.57735923,  4.43207997,  9.74400518],
       [ 3.2331676 , 10.36556   , 11.12098635],
       [ 3.48177234,  3.89311996, 12.97157832],
       [-0.82596913,  9.82657994,  7.89343254],
       [ 1.06269556,  7.97379002,  0.938